# 1. Установка и загрузка библиотек

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Установка необходимых библиотек
!pip install --upgrade --force-reinstall numpy
!pip install -U gensim summa tqdm

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 60.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.0 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.0 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 17.9 MB/s eta 0:00:00
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54387 sha256=88914cf76367415a6644f23a7b8ddc836a13b9833cc0b772a79e112ba887ec34
  Stored in directory: /root/.cache/pip/wheels/10/2d/7a/abce87c4ea233f8dcca0d99b740ac0257eced1f99a124a0e1f
Successfully built summa
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.0
    Uninstalling numpy-2.3.0:
      Successfully uninstalled numpy-2.3.0
  Attempting uninstall: scipy
    Found existing insta

# 2. Перезапуск среды выполнения

In [ ]:
# Среда выполнения → Перезапустить сеанс

# 3. Импорт библиотек

In [ ]:
import pandas as pd
from gensim import corpora, models
from summa import keywords as summa_keywords
from tqdm.notebook import tqdm
import os, gc, csv

# 4. Настройки и путь к файлу Cyberleninka

In [ ]:
path = '/content/drive/MyDrive/TextScope/processed_cyberleninka_data.csv'
out_path = path.replace('.csv', '_keywords.csv')
chunk_size = 500
text_column = 'text'

# 5. Функции извлечения кандидатов

In [ ]:
# Чистка текста: токенизация
def tokenize(text):
    return text.lower().split()

# Топ слов по TF-IDF (gensim)
def top_tfidf_gensim(doc_bow, tfidf_model, dictionary, top_n=30):
    tfidf_scores = tfidf_model[doc_bow]
    tfidf_scores = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)
    top_terms = [dictionary[id] for id, score in tfidf_scores[:top_n]]
    return set(top_terms)

# Топ слов по TextRank (summa)
def top_textrank(text, top_n=30):
    terms = summa_keywords.keywords(text, split=True, scores=False)
    return set(terms[:top_n])

def union_candidates(tfidf_set, tr_set, top_k=30):
    union = list(tfidf_set | tr_set)
    return union[:top_k]

# 6. Главная функция извлечения ключевых слов

In [ ]:
MAX_WORDS = 5000  # максимум слов в тексте

def iter_corpus(path, dictionary):
    reader = pd.read_csv(path, chunksize=chunk_size)
    for chunk in reader:
        texts = [tokenize(str(text)) for text in chunk[text_column]]
        for text in texts:
            if len(text) > MAX_WORDS:
                continue  # пропуск слишком длинных текстов
            yield dictionary.doc2bow(text)

def iter_texts(path):
    reader = pd.read_csv(path, chunksize=chunk_size)
    for chunk in reader:
        texts = [tokenize(str(text)) for text in chunk[text_column]]
        for text in texts:
            if len(text) > MAX_WORDS:
                continue  # пропуск слишком длинных текстов
            yield text

print("Строим словарь...")
dictionary = corpora.Dictionary(iter_texts(path))
dictionary.filter_extremes(no_below=2, no_above=0.85)
print(f"Размер словаря: {len(dictionary)}")

print("Строим TF-IDF модель...")
corpus = iter_corpus(path, dictionary)
tfidf_model = models.TfidfModel(corpus, dictionary=dictionary)

print("Создаем индекс для TF-IDF...")
index_corpus = list(iter_corpus(path, dictionary))
tfidf_corpus = tfidf_model[index_corpus]

print("Извлечение ключевых слов...")

reader = pd.read_csv(path, chunksize=chunk_size)
total_rows = sum(1 for _ in open(path)) - 1  # для прогресса

for chunk_num, chunk in enumerate(tqdm(reader, desc="Обработка чанков")):
    chunk_keywords = []

    for row in tqdm(chunk.itertuples(), desc=f"Чанк {chunk_num + 1}", leave=False):
        text = str(getattr(row, text_column))
        tokens = tokenize(text)
        if len(tokens) > MAX_WORDS:
            chunk_keywords.append([])  # если слишком длинный — пустой список
            continue

        bow = dictionary.doc2bow(tokens)
        tfidf_weights = tfidf_model[bow]

        tfidf_cand = set([dictionary[id] for id, weight in sorted(tfidf_weights, key=lambda x: -x[1])[:30]])
        tr_cand = top_textrank(text)

        final_keywords = union_candidates(tfidf_cand, tr_cand)
        chunk_keywords.append(final_keywords)

    chunk['keywords'] = chunk_keywords
    mode = 'a' if os.path.exists(out_path) else 'w'
    header = not os.path.exists(out_path)
    chunk.to_csv(out_path, mode=mode, index=False, header=header, quoting=csv.QUOTE_ALL, quotechar='"')
    gc.collect()

print(f"Обработка завершена. Результат сохранен в: {out_path}")

Строим словарь...


Размер словаря: 100000
Строим TF-IDF модель...
Создаем индекс для TF-IDF...
Извлечение ключевых слов...


Обработка чанков: 0it [00:00, ?it/s]

Чанк 1: 0it [00:00, ?it/s]

Чанк 2: 0it [00:00, ?it/s]

...

Чанк 43: 0it [00:00, ?it/s]

Чанк 44: 0it [00:00, ?it/s]

Чанк 45: 0it [00:00, ?it/s]

Чанк 46: 0it [00:00, ?it/s]

Обработка завершена. Результат сохранен в: /content/drive/MyDrive/TextScope/processed_cyberleninka_data_keywords.csv
